In [20]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [121]:
data = pd.read_csv('train.csv')
data.head()

,Id,Comment,Topic
0,0x840,A few things. You might have negative- frequen...,Biology
1,0xbf0,Is it so hard to believe that there exist part...,Physics
2,0x1dfc,There are bees,Biology
3,0xc7e,I'm a medication technician. And that's alot o...,Biology
4,0xbba,Cesium is such a pretty metal.,Chemistry


In [122]:
data.shape

(8695, 3)

In [123]:
data.drop(columns='Id', axis=1, inplace=True)

In [124]:
data = data[~data['Comment'].str.contains(r'\[removed\]|\[deleted\]', case=False)]

In [125]:
data['Comment'] = data['Comment'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

C:\Users\ksyuk\AppData\Local\Temp\ipykernel_5952\1186963608.py:1: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  data['Comment'] = data['Comment'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
C:\Users\ksyuk\AppData\Local\Temp\ipykernel_5952\1186963608.py:1: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  data['Comment'] = data['Comment'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())


In [126]:
data['Comment'] = data['Comment'].apply(lambda x: x.lower())

In [127]:
data['Comment'] = data['Comment'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))
data['Comment'] = data['Comment'].apply(lambda x: re.sub(r'\S+@\S+', '', x))

In [128]:
data.head()

,Comment,Topic
0,a few things. you might have negative- frequen...,Biology
1,is it so hard to believe that there exist part...,Physics
2,there are bees,Biology
3,i'm a medication technician. and that's alot o...,Biology
4,cesium is such a pretty metal.,Chemistry


In [129]:
data['Comment'] = data['Comment'].apply(lambda x: re.sub(r"["
    u"\U0001F600-\U0001F64F"  # Эмодзи
    u"\U0001F300-\U0001F5FF"  # Символы и пиктограммы
    u"\U0001F680-\U0001F6FF"  # Символы транспорта и карт
    u"\U0001F1E0-\U0001F1FF"  # Флаги
    u"\U00002500-\U00002BEF"  # Символы, похожие на знаки пунктуации
    u"\U00002702-\U000027B0"
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
    "]+", '', x))

In [130]:
data.head(15)

,Comment,Topic
0,a few things. you might have negative- frequen...,Biology
1,is it so hard to believe that there exist part...,Physics
2,there are bees,Biology
3,i'm a medication technician. and that's alot o...,Biology
4,cesium is such a pretty metal.,Chemistry
5,i meant that the question itself is unclear.,Chemistry
6,shove it up your ass and see what happens,Biology
7,"??? i mean it has some butter, but besides tha...",Chemistry
8,,Biology
9,"well, that’s just the thing. you can’t really ...",Biology


In [131]:
data = data[data['Comment'].str.strip().astype(bool)]

In [132]:
data.head(15)

,Comment,Topic
0,a few things. you might have negative- frequen...,Biology
1,is it so hard to believe that there exist part...,Physics
2,there are bees,Biology
3,i'm a medication technician. and that's alot o...,Biology
4,cesium is such a pretty metal.,Chemistry
5,i meant that the question itself is unclear.,Chemistry
6,shove it up your ass and see what happens,Biology
7,"??? i mean it has some butter, but besides tha...",Chemistry
9,"well, that’s just the thing. you can’t really ...",Biology
10,is h2s what produced when the egg protein is ...,Chemistry


In [133]:
data.shape

(8486, 2)

In [134]:
import string
punct = string.punctuation

print(punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [135]:
punct = '!"#$&\'()*+,-./:;<>?@[\\]^_`{|}~'

In [136]:
data['Comment'] = data['Comment'].str.replace(f"[{punct}]", "", regex=True)

In [137]:
data.head(15)

,Comment,Topic
0,a few things you might have negative frequency...,Biology
1,is it so hard to believe that there exist part...,Physics
2,there are bees,Biology
3,im a medication technician and thats alot of d...,Biology
4,cesium is such a pretty metal,Chemistry
5,i meant that the question itself is unclear,Chemistry
6,shove it up your ass and see what happens,Biology
7,i mean it has some butter but besides that it...,Chemistry
9,well that’s just the thing you can’t really in...,Biology
10,is h2s what produced when the egg protein is ...,Chemistry


In [138]:
data['Topic'] = data['Topic'].replace({'Physics': 0, 'Chemistry': 1, 'Biology': 2})
data['Topic'].value_counts()

C:\Users\ksyuk\AppData\Local\Temp\ipykernel_5952\2510619490.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Topic'] = data['Topic'].replace({'Physics': 0, 'Chemistry': 1, 'Biology': 2})


Topic
2    3563
1    2883
0    2040
Name: count, dtype: int64

In [139]:
from nltk.tokenize import WordPunctTokenizer

data['Comment'] = data['Comment'].apply(WordPunctTokenizer().tokenize)
data.head()

,Comment,Topic
0,"[a, few, things, you, might, have, negative, f...",2
1,"[is, it, so, hard, to, believe, that, there, e...",0
2,"[there, are, bees]",2
3,"[im, a, medication, technician, and, thats, al...",2
4,"[cesium, is, such, a, pretty, metal]",1


In [140]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [141]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def preprocess(tokens):
    return [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words]

data['Comment'] = data['Comment'].apply(preprocess)
data.head()

,Comment,Topic
0,"[thing, might, negative, frequency, dependent,...",2
1,"[hard, believe, exist, particular, cant, detec...",0
2,[bee],2
3,"[im, medication, technician, thats, alot, drug...",2
4,"[cesium, pretty, metal]",1


In [142]:
data.head(15)

,Comment,Topic
0,"[thing, might, negative, frequency, dependent,...",2
1,"[hard, believe, exist, particular, cant, detec...",0
2,[bee],2
3,"[im, medication, technician, thats, alot, drug...",2
4,"[cesium, pretty, metal]",1
5,"[meant, question, unclear]",1
6,"[shove, as, see, happens]",2
7,"[mean, butter, besides, ’, sugar, baking, soda...",1
9,"[well, ’, thing, ’, really, induce, immune, re...",2
10,"[h2s, produced, egg, protein, denatured, titra...",1


In [143]:
punct = string.punctuation + "’‘”“"

def delete_punct(lemmas):
    return [x for x in lemmas if x not in punct]

data['Comment'] = data['Comment'].apply(delete_punct)
data.head()

,Comment,Topic
0,"[thing, might, negative, frequency, dependent,...",2
1,"[hard, believe, exist, particular, cant, detec...",0
2,[bee],2
3,"[im, medication, technician, thats, alot, drug...",2
4,"[cesium, pretty, metal]",1


In [144]:
data.head(15)

,Comment,Topic
0,"[thing, might, negative, frequency, dependent,...",2
1,"[hard, believe, exist, particular, cant, detec...",0
2,[bee],2
3,"[im, medication, technician, thats, alot, drug...",2
4,"[cesium, pretty, metal]",1
5,"[meant, question, unclear]",1
6,"[shove, as, see, happens]",2
7,"[mean, butter, besides, sugar, baking, soda, p...",1
9,"[well, thing, really, induce, immune, response...",2
10,"[h2s, produced, egg, protein, denatured, titra...",1


In [165]:
data['X'] = data['Comment'].apply(lambda x: ' '.join(x))
data.head()

,Comment,Topic,X
0,"[thing, might, negative, frequency, dependent,...",2,thing might negative frequency dependent selec...
1,"[hard, believe, exist, particular, cant, detec...",0,hard believe exist particular cant detect anyt...
2,[bee],2,bee
3,"[im, medication, technician, thats, alot, drug...",2,im medication technician thats alot drug liver...
4,"[cesium, pretty, metal]",1,cesium pretty metal


In [166]:
from collections import Counter
import numpy as np

all_words = [word for comment in data['Comment'] for word in comment]
word_freq = Counter(all_words)

top_10_words = word_freq.most_common(20)

for word, freq in top_10_words:
    print(f"'{word}' встречается {freq} раз")

'like' встречается 1066 раз
'would' встречается 1050 раз
'one' встречается 858 раз
'get' встречается 727 раз
'think' встречается 610 раз
'also' встречается 581 раз
'make' встречается 574 раз
'know' встречается 566 раз
'time' встречается 561 раз
'dont' встречается 508 раз
'thing' встречается 497 раз
'people' встречается 475 раз
'could' встречается 465 раз
'much' встречается 459 раз
'im' встречается 455 раз
'even' встречается 427 раз
'see' встречается 420 раз
'good' встречается 415 раз
'way' встречается 414 раз
'need' встречается 409 раз


In [169]:
def remove_common_words(tokens):
    return [word for word in tokens if word not in top_10_words]

data['Comment'] = data['Comment'].apply(remove_common_words)

In [170]:
data.head()

,Comment,Topic,X
0,"[thing, might, negative, frequency, dependent,...",2,"[t, h, i, n, g, , m, i, g, h, t, , n, e, g, ..."
1,"[hard, believe, exist, particular, cant, detec...",0,"[h, a, r, d, , b, e, l, i, e, v, e, , e, x, ..."
2,[bee],2,"[b, e, e]"
3,"[im, medication, technician, thats, alot, drug...",2,"[i, m, , m, e, d, i, c, a, t, i, o, n, , t, ..."
4,"[cesium, pretty, metal]",1,"[c, e, s, i, u, m, , p, r, e, t, t, y, , m, ..."


In [171]:
data['X'] = data['Comment'].apply(lambda x: ' '.join(x))
data.head()

,Comment,Topic,X
0,"[thing, might, negative, frequency, dependent,...",2,thing might negative frequency dependent selec...
1,"[hard, believe, exist, particular, cant, detec...",0,hard believe exist particular cant detect anyt...
2,[bee],2,bee
3,"[im, medication, technician, thats, alot, drug...",2,im medication technician thats alot drug liver...
4,"[cesium, pretty, metal]",1,cesium pretty metal


In [172]:
data = data[data['X'].str.strip().astype(bool)]
data.shape

(8439, 3)

In [173]:
X_train, X_test, y_train, y_test = train_test_split(data['X'], data['Topic'],
                                                    test_size=0.2, random_state=42, stratify=data['Topic'])

In [184]:
from gensim.models import Word2Vec
import numpy as np

model = Word2Vec(data['X'].apply(lambda x: x.split()), min_count=20, window=6, sample=6e-5, alpha=0.03,
                min_alpha=0.0007, negative=20)

def get_vector(sentence):
    words = sentence.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

X_train_w2v = np.array([get_vector(sentence) for sentence in X_train])
X_test_w2v = np.array([get_vector(sentence) for sentence in X_test])

In [185]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=49, min_samples_split=10, min_samples_leaf=2, random_state=42)
rfc.fit(X_train_w2v, y_train)

RandomForestClassifier(max_depth=49, min_samples_leaf=2, min_samples_split=10,
                       random_state=42)

In [186]:
y_pred = rfc.predict(X_test_w2v)

accuracy_rfc_w2v = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_rfc_w2v:.2f}")

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.50
              precision    recall  f1-score   support

           0       0.59      0.16      0.26       405
           1       0.47      0.39      0.43       572
           2       0.51      0.79      0.62       711

    accuracy                           0.50      1688
   macro avg       0.53      0.45      0.43      1688
weighted avg       0.52      0.50      0.47      1688

[[ 66 115 224]
 [ 28 225 319]
 [ 17 134 560]]


In [187]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train_w2v, y_train)
y_pred_xgb = xgb.predict(X_test_w2v)

accuracy_xgb_w2v = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy with XGBoost: {accuracy_xgb_w2v:.2f}")
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

c:\Users\ksyuk\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [22:45:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy with XGBoost: 0.53
              precision    recall  f1-score   support

           0       0.52      0.30      0.38       405
           1       0.49      0.45      0.47       572
           2       0.56      0.73      0.63       711

    accuracy                           0.53      1688
   macro avg       0.52      0.49      0.49      1688
weighted avg       0.53      0.53      0.52      1688

[[122 113 170]
 [ 73 258 241]
 [ 38 155 518]]


## Glove

In [181]:
def load_glove_model(glove_file):
    glove_model = {}
    with open(glove_file, encoding="utf8") as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    return glove_model

glove_model = load_glove_model('glove.6B.100d.txt')

def get_glove_vector(sentence):
    words = sentence.split()
    vectors = [glove_model[word] for word in words if word in glove_model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(100)

X_train_glove = np.array([get_glove_vector(sentence) for sentence in X_train])
X_test_glove = np.array([get_glove_vector(sentence) for sentence in X_test])

In [182]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=49, min_samples_split=10, min_samples_leaf=2, random_state=42)
rfc.fit(X_train_glove, y_train)

y_pred = rfc.predict(X_test_glove)

accuracy_rfc_glove = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_rfc_glove:.2f}")

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.66
              precision    recall  f1-score   support

           0       0.75      0.43      0.55       405
           1       0.66      0.57      0.61       572
           2       0.64      0.86      0.73       711

    accuracy                           0.66      1688
   macro avg       0.68      0.62      0.63      1688
weighted avg       0.67      0.66      0.65      1688

[[174  86 145]
 [ 40 327 205]
 [ 18  81 612]]


In [183]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb.fit(X_train_glove, y_train)
y_pred_xgb = xgb.predict(X_test_glove)

accuracy_xgb_glove = accuracy_score(y_test, y_pred_xgb)
print(f"Accuracy with XGBoost: {accuracy_xgb_glove:.2f}")
print(classification_report(y_test, y_pred_xgb))
print(confusion_matrix(y_test, y_pred_xgb))

c:\Users\ksyuk\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\core.py:158: UserWarning: [22:42:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy with XGBoost: 0.68
              precision    recall  f1-score   support

           0       0.68      0.56      0.61       405
           1       0.67      0.60      0.63       572
           2       0.68      0.81      0.74       711

    accuracy                           0.68      1688
   macro avg       0.68      0.66      0.66      1688
weighted avg       0.68      0.68      0.67      1688

[[226  75 104]
 [ 64 342 166]
 [ 42  91 578]]
